In [4]:
# Import
import os
import random
import pygame
import mediapipe as mp
import cv2
import numpy as np
from Fruit import Fruit
import pymunk
import time

# Pygame 초기화
pygame.init()
pygame.mixer.init()  # 소리 초기화를 위한 라인 추가
pygame.event.clear()

# 창/화면 생성
width, height = 1200, 686
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Fruit Slicer")

# FPS용 시계 초기화
fps = 23
clock = pygame.time.Clock()

# 이미지
imgGameOver = pygame.image.load(r"C:\Users\bluecom011\Desktop\Fruit\fru.jpg").convert()


# detector = HandDetector(maxHands=1, detectionCon=0.8)
# Mediapipe 포즈 클래스 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 웹캠
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)  # 너비
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)   # 높이

# 메타데이터
# fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create Pygame window with the dimensions of the webcam frames
window = pygame.display.set_mode((width, height))

# 물리 공간 초기화
space = pymunk.Space()
space.gravity = 0.0, -1000.0

# 게임 파라미터
timeTotal = 60

# 게임 변수
fruitList = []
timeGenerator = time.time()
timeStart = time.time()
gameOver = False
score = 0

# 색상 정의
blue = (255, 127, 0)
yellow = (0, 255, 255)
white = (255, 255, 255)
black = (0, 0, 0)

# 과일 이미지 경로 목록
pathFruitFolder = r"C:\Users\bluecom011\Desktop\Fruit/Fruits"
pathListFruit = os.listdir(pathFruitFolder)


def generateFruit():
    randomScale = round(random.uniform(0.6, 0.8), 2)
    randomFruitPath = pathListFruit[random.randint(0, len(pathListFruit) - 1)]
    if "bomb" in randomFruitPath:
        pathSoundSlice = r'C:\Users\bluecom011\Desktop\Fruit/explosion.wav'
    else:
        pathSoundSlice = r'C:\Users\bluecom011\Desktop\Fruit/slice.wav'

    fruitList.append(Fruit(space, path=os.path.join(pathFruitFolder, randomFruitPath),
                           grid=(4, 4), animationFrames=14, scale=randomScale,
                           pathSoundSlice=pathSoundSlice))


def game_loop():
    global gameOver, score, timeGenerator, timeStart

    while cap.isOpened():
        # 이벤트 처리
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        if gameOver is False:
            success, img = cap.read()
            img = cv2.flip(img, 1)
            # 높이와 너비 가져오기
            h, w = img.shape[:2]

            # BGR 이미지를 RGB로 변환
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # 이미지 처리
            keypoints = pose.process(img)

            # RGB 이미지를 다시 BGR로 변환
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

           # 이미지 처리
            lm = keypoints.pose_landmarks
            lmPose = mp_pose.PoseLandmark

            try:
                # 검지 손가락
                index_finger_x = int(lm.landmark[lmPose.RIGHT_INDEX].x * w)
                index_finger_y = int(lm.landmark[lmPose.RIGHT_INDEX].y * h)
            except AttributeError:
                continue

            # Draw circles
            cv2.circle(img, (index_finger_x, index_finger_y), 20, blue, -1)

            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgRGB = np.rot90(imgRGB)
            frame = pygame.surfarray.make_surface(imgRGB).convert()
            frame = pygame.transform.flip(frame, True, False)
            window.blit(frame, (0, 0))

            if time.time() - timeGenerator > 1:
                generateFruit()
                timeGenerator = time.time()

            x, y = index_finger_x, index_finger_y
            for i, fruit in enumerate(fruitList):
                if fruit:
                    fruit.draw(window)
                    checkSlice = fruit.checkSlice(x, y)
                    if checkSlice == 2:
                        gameOver = True
                        pygame.mixer.music.stop()
                    if checkSlice == 1:
                        fruitList[i] = False
                        score += 1

            timeLeft = int(timeTotal - (time.time() - timeStart))
            if timeLeft <= 0:
                gameOver = True
                pygame.mixer.music.stop()

            font = pygame.font.Font(None, 60)
            textScore = font.render(str(score), True, blue)
            textTime = font.render(str(timeLeft), True, blue)
            window.blit(textScore, (225, 35))
            window.blit(textTime, (1100, 38))

        else:
            window.blit(imgGameOver, (0, 0))

            # Text Score
            font = pygame.font.Font(None, 150)
            textLose = font.render("You Lose!", True, (0, 0, 0))
            textYour = font.render("Your Score:", True, (0, 0, 0))
            textScore = font.render(str(score), True, (0, 0, 0))
            window.blit(textLose, (400, 143))
            window.blit(textYour, (350, 243))
            window.blit(textScore, (600, 343))

        # Update Display
        pygame.display.update()
        # Set FPS
        clock.tick(fps)
        space.step(1 / fps)

    pygame.quit()


if __name__ == "__main__":
    game_loop()


In [7]:
# Import
import os
import random
import pygame
import mediapipe as mp
import cv2
import numpy as np
from Fruit import Fruit
import pymunk
import time
# Pygame 초기화
pygame.init()
pygame.mixer.init()  # 소리 초기화를 위한 라인 추가
pygame.event.clear()
# 창/화면 생성
width, height = 1200, 686
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Fruit Slicer")
# FPS용 시계 초기화
fps = 23
clock = pygame.time.Clock()
# 이미지
imgGameOver = pygame.image.load(r"C:\Users\bluecom011\Desktop\Fruit/fru.jpg").convert()
# detector = HandDetector(maxHands=1, detectionCon=0.8)
# Mediapipe 포즈 클래스 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
# 웹캠
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)  # 너비
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)   # 높이
# Parameters
timeTotal = 30  # Set the total game time to 30 seconds
# 메타데이터
# fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# Create Pygame window with the dimensions of the webcam frames
window = pygame.display.set_mode((width, height))
# 물리 공간 초기화
space = pymunk.Space()
space.gravity = 0.0, -1000.0
# 게임 파라미터
timeTotal = 60
# 게임 변수
fruitList = []
timeGenerator = time.time()
timeStart = time.time()
gameOver = False
score = 0
# 색상 정의
blue = (255, 127, 0)
yellow = (0, 255, 255)
white = (255, 255, 255)
black = (0, 0, 0)
# 과일 경로 목록
pathFruitFolder = r"C:\Users\bluecom011\Desktop\Fruit/Fruits"
pathListFruit = os.listdir(pathFruitFolder)
def generateFruit():
    randomScale = round(random.uniform(0.6, 0.8), 2)
    randomFruitPath = pathListFruit[random.randint(0, len(pathListFruit) - 1)]
    if "bomb" in randomFruitPath:
        pathSoundSlice = r'C:\Users\bluecom011\Desktop\Fruit/explosion.wav'
    else:
        pathSoundSlice = r'C:\Users\bluecom011\Desktop\Fruit/slice.wav'
    fruitList.append(Fruit(space, path=os.path.join(pathFruitFolder, randomFruitPath),
                           grid=(4, 4), animationFrames=14, scale=randomScale,
                           pathSoundSlice=pathSoundSlice))
def game_loop():
    global gameOver, score, timeGenerator, timeStart
       # Load background music
    pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")  # Replace with your music file path
    pygame.mixer.music.play(-1)  # -1 means play the music indefinitely
    while cap.isOpened():
        # 이벤트 처리
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
        if gameOver is False:
            success, img = cap.read()
            img = cv2.flip(img, 1)
            # 높이와 너비 가져오기
            h, w = img.shape[:2]
            # BGR 이미지를 RGB로 변환
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # 이미지 처리
            keypoints = pose.process(img)
            # RGB 이미지를 다시 BGR로 변환
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            # 이미지 처리
            lm = keypoints.pose_landmarks
            lmPose = mp_pose.PoseLandmark
            try:
                # 검지 손가락 끝 좌표
                index_finger_tip_x = int(lm.landmark[lmPose.RIGHT_INDEX].x * w)
                index_finger_tip_y = int(lm.landmark[lmPose.RIGHT_INDEX].y * h)
                # Draw circles
                cv2.circle(img, (index_finger_tip_x, index_finger_tip_y), 20, blue, -1)
            except AttributeError:
                continue
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgRGB = np.rot90(imgRGB)
            frame = pygame.surfarray.make_surface(imgRGB).convert()
            frame = pygame.transform.flip(frame, True, False)
            window.blit(frame, (0, 0))
            if time.time() - timeGenerator > 1:
                generateFruit()
                timeGenerator = time.time()
            x, y = index_finger_tip_x, index_finger_tip_y
            for i, fruit in enumerate(fruitList):
                if fruit:
                    fruit.draw(window)
                    checkSlice = fruit.checkSlice(x, y)
                    if checkSlice == 2:
                        gameOver = True
                        pygame.mixer.music.stop()
                    if checkSlice == 1:
                        fruitList[i] = False
                        score += 1
            timeLeft = int(timeTotal - (time.time() - timeStart))
            if timeLeft <= 0:
                gameOver = True
                pygame.mixer.music.stop()
            font = pygame.font.Font(None, 60)
            textScore = font.render(str(score), True, blue)
            textTime = font.render(str(timeLeft), True, blue)
            window.blit(textScore, (225, 35))
            window.blit(textTime, (1100, 38))
        else:
            window.blit(imgGameOver, (0, 0))
            # Text Score
            font = pygame.font.Font(None, 150)
            textLose = font.render("You Lose!", True, (0, 0, 0))
            textYour = font.render("Your Score:", True, (0, 0, 0))
            textScore = font.render(str(score), True, (0, 0, 0))
            window.blit(textLose, (400, 143))
            window.blit(textYour, (350, 243))
            window.blit(textScore, (600, 343))
        # Update Display
        pygame.display.update()
        # Set FPS
        clock.tick(fps)
        space.step(1 / fps)
    pygame.quit()
if __name__ == "__main__":
    game_loop()

손 인식해서 끝부분만 표시

In [3]:
import cv2
from HandTrackingModule import HandDetector

cap = cv2.VideoCapture(0)
cam_w, cam_h = 640, 480
cap.set(3, cam_w)
cap.set(4, cam_h)

detector = HandDetector(detectionCon=0.65, maxHands=1)
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img)
    if hands:
        lmlist = hands[0]['lmList']
        ind_x,ind_y = lmlist[8][0], lmlist[8][1]
        cv2.circle(img,(ind_x,ind_y),5,(0,255,255),2)

    cv2.imshow("Camera Feed", img)

    # 키 입력 확인
    key = cv2.waitKey(1)

    # 'q' 키가 눌리면 종료
    if key == ord('q'):
        break

# 창 닫기
cap.release()
cv2.destroyAllWindows()

In [6]:
# Import
import os
import random
import pygame
import mediapipe as mp
import cv2
import numpy as np
from Fruit import Fruit
from HandTrackingModule import HandDetector  # Assuming you have HandTrackingModule in the same directory
import pymunk
import time

# Pygame 초기화
pygame.init()
pygame.mixer.init()  # 소리 초기화를 위한 라인 추가
pygame.event.clear()

# 창/화면 생성
width, height = 1200, 600
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Fruit Slicer")

# FPS용 시계 초기화
fps = 23
clock = pygame.time.Clock()

# 이미지
imgGameOver = pygame.image.load(r"C:\Users\bluecom011\Desktop\Fruit/fru.jpg").convert()

# Mediapipe 포즈 클래스 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 웹캠
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)  # 너비
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)  # 높이

# 메타데이터
# fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create Pygame window with the dimensions of the webcam frames
window = pygame.display.set_mode((width, height))
# Hand detector 초기화
detector = HandDetector(maxHands=1, detectionCon=0.8)

# 물리 공간 초기화
space = pymunk.Space()
space.gravity = 0.0, -1000.0

# 게임 변수 및 파라미터
timeTotal = 60
fruitList = []
timeGenerator = time.time()
timeStart = time.time()
gameOver = False
score = 0
remainingTime = timeTotal  # Variable to track remaining time

# 색상 정의
blue = (255, 127, 0)

# 과일 경로 목록
pathFruitFolder = r"C:\Users\bluecom011\Desktop\Fruit/Fruits"
pathListFruit = os.listdir(pathFruitFolder)

#과일 생성
def generateFruit():
    randomScale = round(random.uniform(0.6, 0.8), 2)
    randomFruitPath = pathListFruit[random.randint(0, len(pathListFruit) - 1)]
    if "bomb" in randomFruitPath:
        pathSoundSlice = r'C:\Users\bluecom011\Desktop\Fruit/explosion.wav'
    else:
        pathSoundSlice = r'C:\Users\bluecom011\Desktop\Fruit/slice.wav'
    fruitList.append(Fruit(space, path=os.path.join(pathFruitFolder, randomFruitPath),
                           grid=(4, 4), animationFrames=14, scale=randomScale,
                           pathSoundSlice=pathSoundSlice))


def game_loop():
    global gameOver, score, timeGenerator, timeStart, remainingTime
    pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
    pygame.mixer.music.play(-1)
    while cap.isOpened() and remainingTime > 0:  # Check if time is remaining
        # 이벤트 처리
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return

        if gameOver is False:
            success, img = cap.read()
            img = cv2.flip(img, 1)
            h, w = img.shape[:2]
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            keypoints = pose.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            lm = keypoints.pose_landmarks
            lmPose = mp_pose.PoseLandmark
            try:
                index_finger_tip_x = int(lm.landmark[lmPose.RIGHT_INDEX].x * w)
                index_finger_tip_y = int(lm.landmark[lmPose.RIGHT_INDEX].y * h)

                hands, img = detector.findHands(img)
                if hands:
                    lmlist = hands[0]['lmList']
                    ind_x, ind_y = lmlist[8][0], lmlist[8][1]
                    index_finger_tip_x, index_finger_tip_y = ind_x, ind_y

                cv2.circle(img, (index_finger_tip_x, index_finger_tip_y), 20, blue, -1)
            except AttributeError:
                continue

            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            imgRGB = np.rot90(imgRGB)
            frame = pygame.surfarray.make_surface(imgRGB).convert()
            frame = pygame.transform.flip(frame, True, False)
            window.blit(frame, (0, 0))

            if time.time() - timeGenerator > 1:
                generateFruit()
                timeGenerator = time.time()

            x, y = index_finger_tip_x, index_finger_tip_y
            for i, fruit in enumerate(fruitList):
                if fruit:
                    fruit.draw(window, (w, h))
                    checkSlice = fruit.checkSlice(x, y)
                    if checkSlice == 2:
                        gameOver = True
                        pygame.mixer.music.stop()
                    if checkSlice == 1:
                        fruitList[i] = False
                        score += 1

            # Update remaining time
            remainingTime = max(0, timeTotal - int(time.time() - timeStart))

            font = pygame.font.Font(None, 60)
            textScore = font.render(str(score), True, blue)
            textTime = font.render(str(remainingTime), True, blue)  # Display remaining time
            window.blit(textScore, (225, 35))
            window.blit(textTime, (1100, 38))
        else:
            window.blit(imgGameOver, (0, 0))
            font = pygame.font.Font(None, 150)
            textLose = font.render("You Lose!", True, (0, 0, 0))
            textYour = font.render("Your Score:", True, (0, 0, 0))
            textScore = font.render(str(score), True, (0, 0, 0))
            window.blit(textLose, (400, 143))
            window.blit(textYour, (350, 243))
            window.blit(textScore, (600, 343))

        pygame.display.update()
        clock.tick(fps)
        space.step(1 / fps)

    pygame.quit()


if __name__ == "__main__":
    game_loop()


AttributeError: 'NoneType' object has no attribute 'shape'